In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque
from scipy.stats import entropy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from environments import *
from models import *
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
rewards_dict = {}
for i in ['dqn','cnn','ann_lstm','cnn_lstm']:
    env = SBEOS_Environment(window_size=10)
    rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores = train_dqn(env,model_type=i)
    plot_metrics(rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores, avg_window=10)
    rewards_dict[i] = rewards

In [ ]:
plot_reward_comparison(rewards_dict, avg_window=10)

In [ ]:
rewards_dict = {}
for i in ['dqn','cnn','ann_lstm','cnn_lstm']:
    env = SBEOS_Environment(window_size=20)
    rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores = train_dqn(env,model_type=i)
    plot_metrics(rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores, avg_window=10)
    rewards_dict[i] = rewards
plot_reward_comparison(rewards_dict, avg_window=10)

In [ ]:
rewards_dict = {}
for i in ['dqn','cnn','ann_lstm','cnn_lstm']:
    env = SBEDS_Environment(window_size=10)
    rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores = train_dqn(env,model_type=i)
    plot_metrics(rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores, avg_window=10)
    rewards_dict[i] = rewards
plot_reward_comparison(rewards_dict, avg_window=10)

In [ ]:
rewards_dict = {}
for i in ['dqn','cnn','ann_lstm','cnn_lstm']:
    env = SBEDS_Environment(window_size=20)
    rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores = train_dqn(env,model_type=i)
    plot_metrics(rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores, avg_window=10)
    rewards_dict[i] = rewards
plot_reward_comparison(rewards_dict, avg_window=10)

In [ ]:
rewards_dict = {}
for i in ['dqn','cnn','ann_lstm','cnn_lstm']:
    env = SBOS_Environment(window_size=10)
    rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores = train_dqn(env,model_type=i)
    plot_metrics(rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores, avg_window=10)
    rewards_dict[i] = rewards
plot_reward_comparison(rewards_dict, avg_window=10)

In [ ]:
rewards_dict = {}
for i in ['dqn','cnn','ann_lstm','cnn_lstm']:
    env = SBOS_Environment(window_size=20)
    rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores = train_dqn(env,model_type=i)
    plot_metrics(rewards, metrics, epoch_lengths, window_sizes, y_true, y_scores, avg_window=10)
    rewards_dict[i] = rewards
plot_reward_comparison(rewards_dict, avg_window=10)